# Scraping Concerts - Lab

## Introduction

Now that you've seen how to scrape a simple website, it's time to again practice those skills on a full-fledged site!
In this lab, you'll practice your scraping skills on a music website: https://www.residentadvisor.net.
## Objectives

You will be able to:
* Scrape events from a website
* Follow links to those events to retrieve further information
* Clean and store scraped data

## View the Website

For this lab, you'll be scraping the https://www.residentadvisor.net website. Start by navigating to the events page [here](https://www.residentadvisor.net/events) in your browser.

<img src="images/ra.png">

In [1]:
import re
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
#Load the https://www.residentadvisor.net/events page in your browser.
#Exploration; designing/testing function parts
response = requests.get("https://www.residentadvisor.net/events/us/losangeles")
soup = BeautifulSoup(response.content, 'html.parser')

## Open the Inspect Element Feature

Next, open the inspect element feature from your web browser in order to preview the underlying HTML associated with the page.

In [40]:
soup.find('div', id="event-listing").findAll('li')[1].find('h1', class_="event-title")

<h1 class="event-title" itemprop="summary"><a href="/events/1287298" itemprop="url" title="Event details of Temple Tuesdays presents: Niko, John Smith, Subvision">Temple Tuesdays presents: Niko, John Smith, Subvision</a> <span>at <a href="/club.aspx?id=69382">Pattern Bar</a></span></h1>

In [37]:
soup.find('div', id="event-listing").findAll('li')[1].find('h1', class_="event-title").text

'Temple Tuesdays presents: Niko, John Smith, Subvision at Pattern Bar'

In [35]:
#Open the inspect element feature in your browser
event_listings = soup.find('div', id="event-listing")
entries = event_listings.findAll('li')
print(len(entries), entries[0])

53 <li><p class="eventDate date"><a href="/events.aspx?ai=23&amp;v=day&amp;mn=7&amp;yr=2019&amp;dy=9"><span>Tue, 09 Jul 2019 /</span></a></p></li>


In [36]:
#Successive exploration in function development
rows = []
for entry in entries:
    #Is it a date? If so, set current date.
    date = entry.find('p', class_="eventDate date")
    event = entry.find('h1', class_="event-title")
    if event:
        details = event.text.split(' at ')
        event_name = details[0].strip()
        venue = details[1].strip()
        try:
            n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
        except:
            n_attendees = np.nan
        rows.append([event_name, venue, cur_date, n_attendees])
    elif date:
        cur_date = date.text
    else:
        continue
df = pd.DataFrame(rows)
df.head()

,0,1,2,3
0,"Temple Tuesdays presents: Niko, John Smith, Su...",Pattern Bar,"Tue, 09 Jul 2019 /",3.0
1,Focus presents: Dillon Nathaniel,The Circle OC,"Tue, 09 Jul 2019 /",1.0
2,Clinic with Boris (Transmit / Tronic),The Sayers Club,"Wed, 10 Jul 2019 /",17.0
3,Dub Club Presents: Sister Nancy / Mark Ernestu...,The Echoplex,"Wed, 10 Jul 2019 /",22.0
4,Tony Bennett,Hollywood Bowl,"Wed, 10 Jul 2019 /",1.0


## Write a Function to Scrape all of the Events on the Given Page Events Page

The function should return a Pandas DataFrame with columns for the Event_Name, Venue, Event_Date and Number_of_Attendees.

In [41]:
def scrape_events(events_page_url):
    #Your code here
    response = requests.get(events_page_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    entries = event_listings.findAll('li')
    rows = []
    for entry in entries:
        #Is it a date? If so, set current date.
        date = entry.find('p', class_="eventDate date")
        event = entry.find('h1', class_="event-title")
        if event:
            details = event.text.split(' at ')
            event_name = details[0].strip()
            venue = details[1].strip()
            try:
                n_attendees = int(re.match("(\d*)", entry.find('p', class_="attending").text)[0])
            except:
                n_attendees = np.nan
            rows.append([event_name, venue, cur_date, n_attendees])
        elif date:
            cur_date = date.text
        else:
            continue
    df = pd.DataFrame(rows)
    df.head()
    df.columns = ["Event_Name", "Venue", "Event_Date", "Number_of_Attendees"]
    return df

## Write a Function to Retrieve the URL for the Next Page

In [42]:
#Function development cell
soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']

'/events/us/losangeles/week/2019-07-16'

In [43]:
def next_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    url_ext = soup.find('a', attrs={'ga-event-action':"Next "}).attrs['href']
    next_page_url = "https://www.residentadvisor.net" + url_ext
    return next_page_url

## Scrape the Next 1000 Events for Your Area

Display the data sorted by the number of attendees. If there is a tie for the number attending, sort by event date.

In [49]:
dfs = []
urls = []
total_rows = 0
cur_url = "https://www.residentadvisor.net/events/us/losangeles"
while total_rows <= 200:
    df = scrape_events(cur_url)
    dfs.append(df)
    total_rows += len(df)
    cur_url = next_page(cur_url)
    urls.append(cur_url)
    time.sleep(.2)
df = pd.concat(dfs)
df = df.iloc[:200]
print(len(df))
df.head()

200


,Event_Name,Venue,Event_Date,Number_of_Attendees
0,"Temple Tuesdays presents: Niko, John Smith, Su...",Pattern Bar,"Tue, 09 Jul 2019 /",3.0
1,Focus presents: Dillon Nathaniel,The Circle OC,"Tue, 09 Jul 2019 /",1.0
2,Clinic with Boris (Transmit / Tronic),The Sayers Club,"Wed, 10 Jul 2019 /",17.0
3,Dub Club Presents: Sister Nancy / Mark Ernestu...,The Echoplex,"Wed, 10 Jul 2019 /",22.0
4,Tony Bennett,Hollywood Bowl,"Wed, 10 Jul 2019 /",1.0


In [50]:
print(urls)

['https://www.residentadvisor.net/events/us/losangeles/week/2019-07-19', 'https://www.residentadvisor.net/events/us/losangeles/week/2019-07-26', 'https://www.residentadvisor.net/events/us/losangeles/week/2019-08-02', 'https://www.residentadvisor.net/events/us/losangeles/week/2019-08-09', 'https://www.residentadvisor.net/events/us/losangeles/week/2019-08-16']


In [51]:
df.head(50)

,Event_Name,Venue,Event_Date,Number_of_Attendees
0,"Temple Tuesdays presents: Niko, John Smith, Su...",Pattern Bar,"Tue, 09 Jul 2019 /",3.0
1,Focus presents: Dillon Nathaniel,The Circle OC,"Tue, 09 Jul 2019 /",1.0
2,Clinic with Boris (Transmit / Tronic),The Sayers Club,"Wed, 10 Jul 2019 /",17.0
3,Dub Club Presents: Sister Nancy / Mark Ernestu...,The Echoplex,"Wed, 10 Jul 2019 /",22.0
4,Tony Bennett,Hollywood Bowl,"Wed, 10 Jul 2019 /",1.0
5,"Warhol Wednesdays: Chris Herrera, Odalfer Daqu...",The Lexington,"Wed, 10 Jul 2019 /",1.0
6,Jolene: An All Inclusive Womxn Strip Club,Cheetah's,"Thu, 11 Jul 2019 /",16.0
7,Made to Move,General Lee's Cocktail House,"Thu, 11 Jul 2019 /",5.0
8,Reggae on Sunset,Los Globos,"Thu, 11 Jul 2019 /",2.0
9,Vudeux and Careless Records present Protocol,Pattern Bar,"Thu, 11 Jul 2019 /",1.0


## Summary 

Congratulations! In this lab, you successfully scraped a website for concert event information!